In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

## Titanic
data_titanic = pd.read_csv("data/titanic/train.csv")

column_titanic_dict = {"PassengerId": "PassengerId", "Survived": "Survived", "Pclass": "TicketClass", "Name": "Name", "Sex": "Sex", "Age": "Age", "SibSp": "NumberSiblingsSpouses", "Parch": "NumberParentsChildren", "Ticket": "TicketNumber", "Fare": "Fare", "Cabin": "CabinNumber", "Embarked": "Port"}
data_titanic = data_titanic.rename(columns=column_titanic_dict)

train_X_titanic_columns = ["TicketClass", "Sex", "Age", "NumberSiblingsSpouses", "NumberParentsChildren", "Fare", "Port"]
train_y_titanic_columns = ["Survived"]
train_X_titanic_ordinal_columns = ["Sex", "Port"]

X_titanic = data_titanic[train_X_titanic_columns]
y_titanic = data_titanic[train_y_titanic_columns]
 
X_titanic.loc[:, train_X_titanic_ordinal_columns] = OrdinalEncoder().fit_transform(X_titanic[train_X_titanic_ordinal_columns])

X_titanic_train, X_titanic_test, y_titanic_train, y_titanic_test = train_test_split(X_titanic, y_titanic, test_size=0.4)
y_titanic_train = np.squeeze(y_titanic_train)
y_titanic_test = np.squeeze(y_titanic_test)

## Machine Failure
data_machine_failure = pd.read_csv("data/machine_failure/train.csv")

column_machine_failure_dict = {"PassengerId": "PassengerId", "Survived": "Survived", "Pclass": "TicketClass", "Name": "Name", "Sex": "Sex", "Age": "Age", "SibSp": "NumberSiblingsSpouses", "Parch": "NumberParentsChildren", "Ticket": "TicketNumber", "Fare": "Fare", "Cabin": "CabinNumber", "Embarked": "Port"}
data_machine_failure = data_machine_failure.rename(columns=column_machine_failure_dict)

train_X_machine_failure_columns = ["TicketClass", "Sex", "Age", "NumberSiblingsSpouses", "NumberParentsChildren", "Fare", "Port"]
train_y_machine_failure_columns = ["Survived"]
train_X_machine_failure_ordinal_columns = ["Sex", "Port"]

X_machine_failure = data_machine_failure[train_X_machine_failure_columns]
y_machine_failure = data_machine_failure[train_y_machine_failure_columns]
 
X_machine_failure.loc[:, train_X_machine_failure_ordinal_columns] = OrdinalEncoder().fit_transform(X_machine_failure[train_X_machine_failure_ordinal_columns])

X_machine_failure_train, X_machine_failure_test, y_machine_failure_train, y_machine_failure_test = train_test_split(X_machine_failure, y_machine_failure, test_size=0.4)
y_machine_failure_train = np.squeeze(y_machine_failure_train)
y_machine_failure_test = np.squeeze(y_machine_failure_test)

## Synthetic
X_synthetic = np.random.randn(int(3e3)).reshape([int(1e3), 3])
y_synthetic = X_synthetic[:, 0] + X_synthetic[:, 1] + X_synthetic[:, 2] > 0
yn_synthetic = np.logical_not(y_synthetic)

X_synthetic[y_synthetic, :] += np.array([1, 1, 1]) * 0.1
X_synthetic[yn_synthetic, :] -= np.array([1, 1, 1]) * 0.1

X_synthetic_train, X_synthetic_test, y_synthetic_train, y_synthetic_test = train_test_split(X_synthetic, y_synthetic, test_size=0.4)
y_synthetic_train = np.squeeze(y_synthetic_train)
y_synthetic_test = np.squeeze(y_synthetic_test)


/opt/homebrew/anaconda3/envs/kaggle/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/var/folders/3f/d8gl3k3j0bn3wk_cqjvfm31w0000gn/T/ipykernel_13494/3784963557.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_titanic.loc[:, train_X_titanic_ordinal_columns] = OrdinalEncoder().fit_transform(X_titanic[train_X_titanic_ordinal_columns])
/var/folders/3f/d8gl3k3j0bn3wk_cqjvfm31w0000gn/T/ipykernel_13494/3784963557.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [2]:
## Standard Scaler

import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, f1_score, precision_score, accuracy_score, recall_score

param_grid = [
  {"knn_clf__n_neighbors": [5, 10, 15, 20], "knn_clf__weights": ["uniform", "distance"]},
]

knn_clf = GridSearchCV(Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", StandardScaler()),
    ("knn_clf", KNeighborsClassifier()),]),
    param_grid, cv=10, verbose=0)

## Titanic
print("Titanic")
knn_clf.fit(X_titanic_train, y_titanic_train)
y_titanic_knn_clf_pred = knn_clf.predict(X_titanic_test)
# Score
print("Confusion Matrix")
print(confusion_matrix(y_titanic_test, y_titanic_knn_clf_pred))
print("Precision Score")
print(precision_score(y_titanic_test, y_titanic_knn_clf_pred))
print("Accuracy Score")
print(accuracy_score(y_titanic_test, y_titanic_knn_clf_pred))
print("Recall Score")
print(recall_score(y_titanic_test, y_titanic_knn_clf_pred))
print("F1 Score")
print(f1_score(y_titanic_test, y_titanic_knn_clf_pred))
print("Best Estimator")
print(knn_clf.best_estimator_)
print("Best Parameters")
print(knn_clf.best_params_)

print("")
## Machine Failure
print("Machine Failure")
knn_clf.fit(X_machine_failure_train, y_machine_failure_train)
y_machine_failure_knn_clf_pred = knn_clf.predict(X_machine_failure_test)
# Score
print("Confusion Matrix")
print(confusion_matrix(y_machine_failure_test, y_machine_failure_knn_clf_pred))
print("Precision Score")
print(precision_score(y_machine_failure_test, y_machine_failure_knn_clf_pred))
print("Accuracy Score")
print(accuracy_score(y_machine_failure_test, y_machine_failure_knn_clf_pred))
print("Recall Score")
print(recall_score(y_machine_failure_test, y_machine_failure_knn_clf_pred))
print("F1 Score")
print(f1_score(y_machine_failure_test, y_machine_failure_knn_clf_pred))
print("Best Estimator")
print(knn_clf.best_estimator_)
print("Best Parameters")
print(knn_clf.best_params_)

print("")
## Synthetic
print("Synthetic")
knn_clf.fit(X_synthetic_train, y_synthetic_train)
y_synthetic_knn_clf_pred = knn_clf.predict(X_synthetic_test)
# Score
print("Confusion Matrix")
print(confusion_matrix(y_synthetic_test, y_synthetic_knn_clf_pred))
print("Precision Score")
print(precision_score(y_synthetic_test, y_synthetic_knn_clf_pred))
print("Accuracy Score")
print(accuracy_score(y_synthetic_test, y_synthetic_knn_clf_pred))
print("Recall Score")
print(recall_score(y_synthetic_test, y_synthetic_knn_clf_pred))
print("F1 Score")
print(f1_score(y_synthetic_test, y_synthetic_knn_clf_pred))
print("Best Estimator")
print(knn_clf.best_estimator_)
print("Best Parameters")
print(knn_clf.best_params_)

Titanic
Confusion Matrix
[[176  30]
 [ 49 102]]
Precision Score
0.7727272727272727
Accuracy Score
0.7787114845938375
Recall Score
0.6754966887417219
F1 Score
0.7208480565371024
Best Estimator
Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('knn_clf', KNeighborsClassifier(n_neighbors=10))])
Best Parameters
{'knn_clf__n_neighbors': 10, 'knn_clf__weights': 'uniform'}

Machine Failure
Confusion Matrix
[[208  16]
 [ 47  86]]
Precision Score
0.8431372549019608
Accuracy Score
0.8235294117647058
Recall Score
0.6466165413533834
F1 Score
0.7319148936170213
Best Estimator
Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('knn_clf', KNeighborsClassifier(n_neighbors=20))])
Best Parameters
{'knn_clf__n_neighbors': 20, 'knn_clf__weights': 'uniform'}

Synthetic
Confusion Matrix
[[199   0]
 [  1 200]]
Precision Score
1.0
Accuracy Score
0.9975
Recall Score
0.9950248756218906
F1 Score
0.9975062344139651
Best Estimator

In [3]:
## Min Max Scaler

import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, f1_score, precision_score, accuracy_score, recall_score

param_grid = [
  {"knn_clf__n_neighbors": [5, 10, 15, 20], "knn_clf__weights": ["uniform", "distance"]},
]

knn_clf = GridSearchCV(Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", MinMaxScaler()),
    ("knn_clf", KNeighborsClassifier()),]),
    param_grid, cv=10, verbose=0)

## Titanic
print("Titanic")
knn_clf.fit(X_titanic_train, y_titanic_train)
y_titanic_knn_clf_pred = knn_clf.predict(X_titanic_test)
# Score
print("Confusion Matrix")
print(confusion_matrix(y_titanic_test, y_titanic_knn_clf_pred))
print("Precision Score")
print(precision_score(y_titanic_test, y_titanic_knn_clf_pred))
print("Accuracy Score")
print(accuracy_score(y_titanic_test, y_titanic_knn_clf_pred))
print("Recall Score")
print(recall_score(y_titanic_test, y_titanic_knn_clf_pred))
print("F1 Score")
print(f1_score(y_titanic_test, y_titanic_knn_clf_pred))
print("Best Estimator")
print(knn_clf.best_estimator_)
print("Best Parameters")
print(knn_clf.best_params_)

print("")
## Machine Failure
print("Machine Failure")
knn_clf.fit(X_machine_failure_train, y_machine_failure_train)
y_machine_failure_knn_clf_pred = knn_clf.predict(X_machine_failure_test)
# Score
print("Confusion Matrix")
print(confusion_matrix(y_machine_failure_test, y_machine_failure_knn_clf_pred))
print("Precision Score")
print(precision_score(y_machine_failure_test, y_machine_failure_knn_clf_pred))
print("Accuracy Score")
print(accuracy_score(y_machine_failure_test, y_machine_failure_knn_clf_pred))
print("Recall Score")
print(recall_score(y_machine_failure_test, y_machine_failure_knn_clf_pred))
print("F1 Score")
print(f1_score(y_machine_failure_test, y_machine_failure_knn_clf_pred))
print("Best Estimator")
print(knn_clf.best_estimator_)
print("Best Parameters")
print(knn_clf.best_params_)

print("")
## Synthetic
print("Synthetic")
knn_clf.fit(X_synthetic_train, y_synthetic_train)
y_synthetic_knn_clf_pred = knn_clf.predict(X_synthetic_test)
# Score
print("Confusion Matrix")
print(confusion_matrix(y_synthetic_test, y_synthetic_knn_clf_pred))
print("Precision Score")
print(precision_score(y_synthetic_test, y_synthetic_knn_clf_pred))
print("Accuracy Score")
print(accuracy_score(y_synthetic_test, y_synthetic_knn_clf_pred))
print("Recall Score")
print(recall_score(y_synthetic_test, y_synthetic_knn_clf_pred))
print("F1 Score")
print(f1_score(y_synthetic_test, y_synthetic_knn_clf_pred))
print("Best Estimator")
print(knn_clf.best_estimator_)
print("Best Parameters")
print(knn_clf.best_params_)

Titanic
Confusion Matrix
[[170  36]
 [ 46 105]]
Precision Score
0.7446808510638298
Accuracy Score
0.7703081232492998
Recall Score
0.695364238410596
F1 Score
0.7191780821917808
Best Estimator
Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', MinMaxScaler()),
                ('knn_clf',
                 KNeighborsClassifier(n_neighbors=15, weights='distance'))])
Best Parameters
{'knn_clf__n_neighbors': 15, 'knn_clf__weights': 'distance'}

Machine Failure
Confusion Matrix
[[212  12]
 [ 55  78]]
Precision Score
0.8666666666666667
Accuracy Score
0.8123249299719888
Recall Score
0.5864661654135338
F1 Score
0.6995515695067265
Best Estimator
Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', MinMaxScaler()),
                ('knn_clf', KNeighborsClassifier(n_neighbors=20))])
Best Parameters
{'knn_clf__n_neighbors': 20, 'knn_clf__weights': 'uniform'}

Synthetic
Confusion Matrix
[[199   0]
 [  0 201]]
Precision Score
1.0
Accuracy Score
1.0
Recall Score
1.0
F1 Score
1.0
Best Estimator